<a href="https://colab.research.google.com/github/Sakinat-Folorunso/CMP_805_Advanced_Programming_Languages/blob/main/notebooks/CMP805_Week12_PH_Python_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMP805 — Week 12 Practical (Python, Colab)
**Topic:** Tutorial, metaprogramming/macros; project demos & revision**  
**Course:** Advanced Programming Languages (M.Sc.), OOU — CMP805

**Instructor:** **DR SAKINAT FOLORUNSO – ASSOCIATE PROFESSOR OF AI SYSTEMS AND FAIR DATA**  
**Department:** **COMPUTER SCIENCES, OLABISI ONABANJO UNIVERSITY, AGO‑IWOYE, OGUN STATE, NIGERIA**

> In this final lab you’ll implement a **tiny macro expander** over the Week‑1/3 AST (expressions only), practice **hygiene** with `gensym`, and complete a short **revision set**. You’ll also prepare a brief **project demo** checklist.

### Learning goals (≈60 minutes)
- Understand the idea of **macros as AST‑to‑AST transforms** (expansion before evaluation).  
- Implement a **macro registry** and write a few example macros (`when`, `unless`, `let*`, `or`).  
- Avoid **variable capture** with a simple `gensym`‑based hygiene helper.  
- Practice revision problems spanning the course.

In [ ]:
# 🧑‍🎓 Student info
STUDENT_NAME = "Type your full name here"
STUDENT_ID   = "Matric/ID here"
print("Student:", STUDENT_NAME, "| ID:", STUDENT_ID)

In [ ]:
# ✅ Environment check
import sys
major, minor = sys.version_info[:2]
assert (major, minor) >= (3, 10), f"Need Python 3.10+, found {major}.{minor}"
print(f"Python {major}.{minor} OK — match/case available.")

In [ ]:
# =====================================
# Part 1 — Core AST (expressions only) + pretty printer + evaluator
# =====================================
from __future__ import annotations
from dataclasses import dataclass
from typing import Union, Dict, List

@dataclass(frozen=True) class Int:  n: int
@dataclass(frozen=True) class Bool: b: bool
@dataclass(frozen=True) class Var:  x: str
@dataclass(frozen=True) class Add:  a: "Expr"; b: "Expr"
@dataclass(frozen=True) class Sub:  a: "Expr"; b: "Expr"
@dataclass(frozen=True) class Mul:  a: "Expr"; b: "Expr"
@dataclass(frozen=True) class Eq:   a: "Expr"; b: "Expr"
@dataclass(frozen=True) class If:   c: "Expr"; t: "Expr"; e: "Expr"
@dataclass(frozen=True) class Let:  x: str; e1: "Expr"; e2: "Expr"

# Macro node (only present pre-expansion)
@dataclass(frozen=True) class Macro: name: str; args: List["Expr"]

Expr = Int | Bool | Var | Add | Sub | Mul | Eq | If | Let | Macro

def pretty(e: Expr) -> str:
    match e:
        case Int(n):  return str(n)
        case Bool(b): return str(b).lower()
        case Var(x):  return x
        case Add(a,b):return f"({pretty(a)} + {pretty(b)})"
        case Sub(a,b):return f"({pretty(a)} - {pretty(b)})"
        case Mul(a,b):return f"({pretty(a)} * {pretty(b)})"
        case Eq(a,b): return f"({pretty(a)} == {pretty(b)})"
        case If(c,t,e):return f"if {pretty(c)} then {pretty(t)} else {pretty(e)}"
        case Let(x,e1,e2):return f"let {x} = {pretty(e1)} in {pretty(e2)}"
        case Macro(n,args): return f"{n}(" + ", ".join(pretty(a) for a in args) + ")"
        case _: return str(e)

# Big-step eval for Expr (after macro expansion)
Value = Union[int,bool]
Env = Dict[str, Value]

class RuntimeErrorPL(Exception): ...

def eval_expr(e: Expr, env: Env|None=None) -> Value:
    env = {} if env is None else dict(env)
    match e:
        case Int(n): return n
        case Bool(b): return b
        case Var(x):
            if x in env: return env[x]
            raise RuntimeErrorPL(f"unbound {x}")
        case Add(a,b): return eval_expr(a,env) + eval_expr(b,env)
        case Sub(a,b): return eval_expr(a,env) - eval_expr(b,env)
        case Mul(a,b): return eval_expr(a,env) * eval_expr(b,env)
        case Eq(a,b):  return eval_expr(a,env) == eval_expr(b,env)
        case If(c,t,e2): return eval_expr(t,env) if eval_expr(c,env) else eval_expr(e2,env)
        case Let(x,e1,e2):
            v1 = eval_expr(e1,env); env2 = dict(env); env2[x]=v1; return eval_expr(e2,env2)
        case Macro(_,_):
            raise RuntimeErrorPL("evaluate after calling macro_expand()")

In [ ]:
# =====================================
# Part 2 — Macro registry + hygiene via gensym + expander
# =====================================
from typing import Callable, Dict

MacroFn = Callable[[list[Expr]], Expr]
MACROS: Dict[str, MacroFn] = {}

def defmacro(name: str):
    def deco(fn: MacroFn):
        MACROS[name] = fn
        return fn
    return deco

# A tiny gensym helper (fresh variable names to avoid capture)
_gensym_i = 0
def gensym(prefix: str = "g") -> str:
    global _gensym_i
    _gensym_i += 1
    return f"{prefix}_{_gensym_i}"

def macro_expand(e: Expr) -> Expr:
    # recursively expand macros until none remain
    match e:
        case Macro(name, args) if name in MACROS:
            # First expand arguments (common style in Racket's syntax-case expands inside out)
            ex_args = [macro_expand(a) for a in args]
            return macro_expand(MACROS[name](ex_args))
        case Macro(name, args):
            raise RuntimeErrorPL(f"unknown macro {name}")
        case Int(_) | Bool(_) | Var(_):
            return e
        case Add(a,b): return Add(macro_expand(a), macro_expand(b))
        case Sub(a,b): return Sub(macro_expand(a), macro_expand(b))
        case Mul(a,b): return Mul(macro_expand(a), macro_expand(b))
        case Eq(a,b):  return Eq( macro_expand(a), macro_expand(b) )
        case If(c,t,ee): return If(macro_expand(c), macro_expand(t), macro_expand(ee))
        case Let(x,e1,e2): return Let(x, macro_expand(e1), macro_expand(e2))

# Convenience smart constructor for macros in this lab
def M(name: str, *args: Expr) -> Macro:
    return Macro(name, list(args))

In [ ]:
# =====================================
# Part 3 — Define a few macros
# =====================================

@defmacro("when")
def _m_when(args: list[Expr]) -> Expr:
    # when(c, t)  ==>  if c then t else 0
    (c, t) = args
    return If(c, t, Int(0))

@defmacro("unless")
def _m_unless(args: list[Expr]) -> Expr:
    # unless(c, t) ==> if c then 0 else t
    (c, t) = args
    return If(c, Int(0), t)

@defmacro("let*")
def _m_letstar(args: list[Expr]) -> Expr:
    # let*([(x1,e1), (x2,e2), ...], body) => nested lets
    # We encode pairs as consecutive Macro("pair", [Var("x"), e]) nodes for simplicity
    binds_list, body = args
    # binds_list is built using Pair nodes (defined below) or plain Lets; to keep simple,
    # we expect binds_list to be a Python-style list encoded via nested Pair / Nil:
    def to_py_list(lst: Expr):
        out = []
        while isinstance(lst, Pair):
            out.append(lst.elem)
            lst = lst.rest
        if not isinstance(lst, Nil):
            raise RuntimeErrorPL("let*: expected a list of pairs")
        return out
    out = body
    for bind in reversed(to_py_list(binds_list)):
        if not isinstance(bind, Bind): raise RuntimeErrorPL("let*: elements must be Bind(x, expr)")
        out = Let(bind.name, bind.expr, out)
    return out

@defmacro("or")
def _m_or(args: list[Expr]) -> Expr:
    # or(a, b)  ==>  let tmp = a in if tmp then tmp else b   (hygienic: tmp fresh)
    (a, b) = args
    tmp = gensym("tmp")
    return Let(tmp, a, If(Var(tmp), Var(tmp), b))

In [ ]:
# Helper dataclasses to conveniently build lists of bindings for let*
@dataclass(frozen=True) class Nil: pass
@dataclass(frozen=True) class Pair: elem: "Expr"; rest: "Expr"
@dataclass(frozen=True) class Bind: name: str; expr: "Expr"

def L(*elements: Expr) -> Expr:
    """Encode a Python list [e1,e2,...] as Pair(...Pair(eN, Nil()))"""
    out: Expr = Nil()
    for el in reversed(elements):
        out = Pair(el, out)
    return out

In [ ]:
# =====================================
# Part 4 — Demo & self-checks
# =====================================

# 1) when/unless
ex1 = M("when", Eq(Int(1), Int(1)), Add(Int(2), Int(3)))         # when(1==1, 2+3) -> 5
ex2 = M("unless", Eq(Int(1), Int(2)), Mul(Int(4), Int(5)))       # unless(1==2, 4*5) -> 20
for name, e in [("when", ex1), ("unless", ex2)]:
    e_exp = macro_expand(e)
    print(f"{name}: {pretty(e)}  ⇒expand⇒  {pretty(e_exp)}  ⇒eval⇒  {eval_expr(e_exp,{})}")

# 2) let* — nested lets
ex3 = M("let*", L(Bind("x", Int(2)), Bind("y", Add(Var("x"), Int(3)))), Mul(Var("x"), Var("y")))
print("let* before:", pretty(ex3))
ex3_exp = macro_expand(ex3)
print("let* expand:", pretty(ex3_exp), "| value =", eval_expr(ex3_exp, {}))

# 3) or — hygiene (fresh tmp)
a = Eq(Int(1), Int(1))
b = Add(Int(10), Int(1))
ex4 = M("or", a, b)
ex4_exp = macro_expand(ex4)
print("or expand:", pretty(ex4_exp), "| value =", eval_expr(ex4_exp, {}))

print("All basic macro tests passed.")

### 🧪 Your Turn (15–20 minutes)
1. Add a macro `and(a,b)` using a hygienic temporary (short‑circuiting).  
2. Add a macro `cond((c1,e1), (c2,e2), ..., (else, eK))` that expands to nested `if`s. You may encode the list with `L(...)`.  
3. **(Optional)** Implement a small constant‑folder `fold(e)` and show `eval_expr(e) == eval_expr(fold(e))` on at least 5 examples.

### 🎓 Project demo checklist (5–7 minutes per team)
- **Problem statement** (1 sentence) and target **language feature(s)**.  
- **Design sketch**: grammar, key AST forms, and one core rule (typing or evaluation).  
- **Demo**: run one non‑trivial example; show a test case.  
- **Reflection**: one challenge + one thing you’d improve next.  
- **Attribution**: list external sources/libraries (if any).

### 📚 Revision prompts
1) Small‑step vs big‑step: when would you prefer each?  
2) Progress & preservation: give the intuition in one sentence each.  
3) Parametric vs ad‑hoc vs subtyping polymorphism — give one example each.  
4) Why does naive reference counting leak on cycles; how does tracing GC avoid it?  
5) Explain why `Square` as a subclass of `Rectangle` can violate **LSP**, and how composition fixes it.  
6) When does a **JIT** help more than an interpreter/VM, and why?

In [ ]:
# 📝 Save small submission bundle
import json, time
stamp = time.strftime("%Y-%m-%d %H:%M:%S")
submission = {
  "student_name": STUDENT_NAME,
  "student_id": STUDENT_ID,
  "timestamp": stamp,
  "checks": ["when/unless", "let*", "or"],
  "reflection": "(fill in here)"
}
with open("week12_submission.json", "w") as f:
  json.dump(submission, f, indent=2)
print("Saved week12_submission.json — upload with your notebook.")